# [LAB11] 지도학습 > 추천시스템 > SVD

## #01. SVD 알고리즘 개요

- SVD(Singular Value Decomposition)는 잠재요인(Latent Factor)을 학습하는 행렬분해 기반 협업필터링 알고리즘
- KNN이 **유사도 기반 모델**이라면 SVD는 **잠재요인 기반 모델**
- 관측된 평점 행렬을 저차원 잠재공간으로 분해하여 사용자와 아이템의 숨겨진 특성을 학습함.
- Netflix Prize 기간 동안 Simon Funk 가 대중화함

---

### [1] SVD의 기본 개념

사용자-아이템 평점 행렬 R을 저차원 행렬 두 개로 분해함.

$$R \approx PQ^T$$

| 기호 | 의미 |
|------|------|
| R | 사용자-아이템 평점 행렬 |
| P | 사용자 잠재요인 행렬 |
| Q | 아이템 잠재요인 행렬 |
| k | 잠재요인 차원 수 |

- 사용자도 벡터
- 아이템도 벡터
- 두 벡터의 내적(dot product)으로 평점 예측

---

### [2] 예측 공식

Surprise 라이브러리 기준 SVD 예측식은 다음과 같음.

$$\hat{r}_{ui} = \mu + b_u + b_i + q_i^T p_u$$

| 기호 | 의미 |
|------|------|
| μ | 전체 평균 평점 |
| b_u | 사용자 편향 |
| b_i | 아이템 편향 |
| p_u | 사용자 잠재요인 벡터 |
| q_i | 아이템 잠재요인 벡터 |

> → Baseline + 잠재요인 내적 구조

---

### [3] 학습 방식

목표는 다음 손실함수를 최소화하는 것임.

$$\min \sum (r_{ui} - \hat{r}_{ui})^2 + \lambda(||p_u||^2 + ||q_i||^2)$$

| 요소 | 의미 |
|------|------|
| MSE | 예측 오차 최소화 |
| λ | 정규화 계수 |
| ∥p∥² | 과적합 방지 목적 |

학습은 SGD(Stochastic Gradient Descent)로 수행됨.

---

### [4] 해석 예시

사용자 A가 영화 X를 보지 않았다고 가정.

- A는 "액션 선호 + 로맨스 비선호" 잠재 벡터 보유
- X는 "액션 강함 + 로맨스 약함" 특성 벡터 보유
- 두 벡터의 내적 결과가 높게 계산됨

→ 예측 평점 4.6

**즉, 숨겨진 취향과 콘텐츠 특성의 상호작용 결과임**

---

### [5] SVD의 의미

- 잠재요인 기반 협업필터링
- 희소 데이터에 강함
- 대규모 데이터에 적합
- 설명 가능성은 낮음 (직관적 해석 어려움)
- 하이퍼파라미터 튜닝 중요 (n_factors, reg, lr 등)

---

### [6] BaselineOnly vs KNNBasic vs SVD 비교

| 구분 | BaselineOnly | KNNBasic | SVD |
|------|-------------|----------|-----|
| 핵심 개념 | 편향 보정 | 유사도 기반 | 잠재요인 기반 |
| 공식 구조 | μ + b_u + b_i | 유사도 가중 평균 | μ + b_u + b_i + q_i^T p_u |
| 개인 성향 반영 | O | 간접적 | O (벡터화) |
| 이웃 정보 활용 | X | O | X (직접적 이웃 아님) |
| 희소 데이터 대응 | 약함 | 약함 | 강함 |
| 복잡도 | 낮음 | 중간 | 높음 |
| 설명 가능성 | 중간 | 높음 | 낮음 |

### 🔑 정리
- Baseline → "기본 성향 보정"
- KNNBasic → "비슷한 사람을 참고"
- SVD → "숨겨진 취향 벡터를 학습"

## #02. 준비작업

### [1] 패키지 참조

In [1]:
from hossam import *
from pandas import DataFrame, merge

from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

### [2] 데이터 가져오기

In [2]:
origin = load_data("ml100k-ratings")
metadata = load_data("ml100k-metadata")

943명의 사용자가 1,682편의 영화에 대해 남긴 100,000개의 평점 기록으로 구성된 명시적 평가 기반 추천 시스템 학습용 데이터셋 (출처: University of Minnesota)

컬럼명     의미
---------  ---------
user_id    사용자 ID
item_id    아이템 ID
rating     평점
timestamp  평가 시각

ml100k-ratings에 포함된 영화 제목, 공개시기, 장르 정보를 담고 있는 데이터 (출처: University of Minnesota)


## #04. SVD 모델 적합

### [1] DataFrame을 Dataset 객체로 변환

#### 평점의 범위 확인

In [3]:
rating_min = origin["rating"].min()
rating_max = origin["rating"].max()
print(f"Rating 범위: {rating_min} ~ {rating_max}")

Rating 범위: 1 ~ 5


#### surprise 라이브러리에서 사용할 수 있도록 데이터셋을 변환

In [4]:
# 평점의 범위를 지정하여 Reader 객체 생성
reader = Reader(rating_scale=(rating_min, rating_max))

# Dataset 객체를 생성 - 사용자 식별자, 아이템 식별자, 평점만으로 구성된 데이터 구조가 필요하다.
data = Dataset.load_from_df(origin[["user_id", "item_id", "rating"]], reader)
data

### [2] SVD 모델의 주요 하이퍼 파라미터

필수 튜닝 대상은 `n_factors`, `lr_all`, `reg_all` 3가지임.

| 파라미터명 | 핵심도 | 설명 | 기본값 | GridSearchCV 권장값 |
|-----------|--------|------|--------|--------------------|
| n_factors | ⭐⭐⭐ | 잠재요인 차원 수 (사용자·아이템 벡터 크기) | 100 | [20, 50, 100, 150, 200] |
| n_epochs | ⭐⭐ | 전체 데이터 반복 학습 횟수 | 20 | [20, 30, 50, 80] |
| lr_all | ⭐⭐⭐ | 학습률 (SGD step size) | 0.005 | [0.002, 0.005, 0.01] |
| reg_all | ⭐⭐⭐ | 정규화 강도 (과적합 제어) | 0.02 | [0.01, 0.02, 0.05, 0.1] |
| biased | ⭐⭐ | 편향항(μ + b_u + b_i) 포함 여부 | True | [True, False] |
| verbose | ✅ | 학습 로그 출력 여부 | False | [True, False] |

**핵심 파라미터 의미 정리**
- n_factors → 모델 표현력 결정 (클수록 복잡)
- lr_all → 수렴 속도 결정
- reg_all → 과적합 제어
- n_epochs → 충분한 학습 여부 결정

**SVD는 모델 복잡도(n_factors)와 규제(reg_all)의 균형이 성능을 좌우함.**

In [5]:
# 하이퍼파라미터 그리드 (SVD용)
param_grid = {
    "n_factors": [50, 100, 150, 200],
    "n_epochs": [20, 30, 50],
    "lr_all": [0.002, 0.005, 0.01],
    "reg_all": [0.01, 0.02, 0.05, 0.1],
    "biased": [True, False],
}

# RandomizedSearchCV 객체 생성
gs = RandomizedSearchCV(
    SVD,
    param_grid,
    measures=["rmse", "mae"],
    cv=5,
    n_jobs=-1,
    random_state=52,  # 재현성 확보
)

# 학습
gs.fit(data)

# 최적 결과 출력
print(" Best RMSE Score:", gs.best_score["rmse"])
print(" Best Parameters:", gs.best_params["rmse"])

 Best RMSE Score: 0.9127536766781968
 Best Parameters: {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1, 'biased': True}


## #04. 성능평가

### [1] 훈련, 검증 데이터 분리

In [6]:
# 데이터를 학습용과 테스트용으로 분할 (80% 학습, 20% 테스트)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=52)

# 학습용과 테스트용 데이터의 크기를 출력
print(f"Trainset 크기: {train_data.n_ratings}개")
print(f"Testset 크기: {len(test_data)}개")

Trainset 크기: 80000개
Testset 크기: 20000개


### [2] 최적 모델 재학습

In [7]:
# 최적 파라미터 추출
best_params = gs.best_params["rmse"]

# 모델 생성
best_model = SVD(**best_params)

# 전체 데이터 학습
best_model.fit(train_data)

### [3] 예측값 생성

In [8]:
predictions = best_model.test(test_data)
predictions[:5]  # 예측 결과의 일부를 출력

[Prediction(uid=303, iid=679, r_ui=2.0, est=3.0264413551376252, details={'was_impossible': False}),
 Prediction(uid=308, iid=163, r_ui=4.0, est=3.7498368049239765, details={'was_impossible': False}),
 Prediction(uid=327, iid=663, r_ui=4.0, est=3.6882221541028812, details={'was_impossible': False}),
 Prediction(uid=912, iid=479, r_ui=4.0, est=4.091904761686306, details={'was_impossible': False}),
 Prediction(uid=224, iid=329, r_ui=3.0, est=2.5858347575789944, details={'was_impossible': False})]

### [4] 성능평가 지표 생성

KNNBasic과 동일한 코드

**KNN과 SVD가 다른 것은 모델 구조이지 평가 방법은 동일한 지도학습 체계**

> 추천 시스템의 목표는 "정확한 예측"이 아니라 "좋은 추천"에 있기 때문에 RMSE가 낮거나 과적합이 발생했다고 해서 그 모델이 나쁘다는 것은 아니다.  
> "예측 오차 최소화"와 "추천 품질 최적화"는 항상 같지 않다.

In [9]:
cv_rmse = gs.best_score["rmse"]

train_pred = best_model.test(train_data.build_testset())
test_pred = best_model.test(test_data)

train_rmse = accuracy.rmse(train_pred, verbose=False)
train_mae = accuracy.mae(train_pred, verbose=False)

test_rmse = accuracy.rmse(test_pred, verbose=False)
test_mae = accuracy.mae(test_pred, verbose=False)

rmse_gap_train = test_rmse - train_rmse
rmse_gap_cv = test_rmse - cv_rmse
mae_gap = test_mae - train_mae

# 과적합 판정 기준
if rmse_gap_train > 0.05:
    overfit_flag = "과적합 의심"
else:
    overfit_flag = "정상"

result_df = DataFrame(
    {
        "Model": ["SVD"],
        "CV_RMSE": [cv_rmse],
        "Train_RMSE": [train_rmse],
        "Test_RMSE": [test_rmse],
        "RMSE_Gap(Test-Train)": [rmse_gap_train],
        "RMSE_Gap(Test-CV)": [rmse_gap_cv],
        "Train_MAE": [train_mae],
        "Test_MAE": [test_mae],
        "MAE_Gap": [mae_gap],
        "Overfitting": [overfit_flag],
    }
)

result_df

,Model,CV_RMSE,Train_RMSE,Test_RMSE,RMSE_Gap(Test-Train),RMSE_Gap(Test-CV),Train_MAE,Test_MAE,MAE_Gap,Overfitting
0,SVD,0.913,0.728,0.916,0.189,0.004,0.580,0.725,0.145,과적합 의심


## #05. TopN 추천

### [1] 아직 평가하지 않은 아이템에 대한 예측 수행

#### 예측결과 생성

In [10]:
anti_testset = train_data.build_anti_testset()
predictions = best_model.test(anti_testset)
predictions[:5]  # 예측 결과의 일부를 출력

[Prediction(uid=234, iid=205, r_ui=3.5317375, est=3.569084075386446, details={'was_impossible': False}),
 Prediction(uid=234, iid=504, r_ui=3.5317375, est=3.482503010773537, details={'was_impossible': False}),
 Prediction(uid=234, iid=73, r_ui=3.5317375, est=2.947876580534275, details={'was_impossible': False}),
 Prediction(uid=234, iid=475, r_ui=3.5317375, est=3.3069245642533556, details={'was_impossible': False}),
 Prediction(uid=234, iid=294, r_ui=3.5317375, est=2.6849597495805724, details={'was_impossible': False})]

#### 예측결과 데이터프레임 구성

In [11]:
pred_df = DataFrame(
    predictions, columns=["user_id", "item_id", "true_rating", "pred_rating", "details"]
)
pred_df.head()

,user_id,item_id,true_rating,pred_rating,details
0,234,205,3.532,3.569,{'was_impossible': False}
1,234,504,3.532,3.483,{'was_impossible': False}
2,234,73,3.532,2.948,{'was_impossible': False}
3,234,475,3.532,3.307,{'was_impossible': False}
4,234,294,3.532,2.685,{'was_impossible': False}


### [3] 특정 사용자에 대한 상위 10개의 추천 영화 검색

#### 76번 사용자에 대한 Top 10 추천 데이터

In [12]:
N = 10
user_id = 76

topn_df = pred_df[pred_df["user_id"] == user_id]

topn_df = (
    topn_df[["user_id", "item_id", "pred_rating"]]
    .sort_values(["pred_rating"], ascending=[False])
    .groupby("user_id")
    .head(N)
    .reset_index(drop=True)
)

topn_df

,user_id,item_id,pred_rating
0,76,1449,4.428
1,76,483,4.348
2,76,654,4.343
3,76,647,4.339
4,76,1512,4.302
5,76,127,4.278
6,76,408,4.276
7,76,187,4.262
8,76,114,4.253
9,76,285,4.250
